In [ ]:

# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-6311  GCE       4                                       RUNNING  us-central1-a


In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:

# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  9 10:30 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:

spark

In [ ]:

# Put your bucket name below and make sure you can access it without an error
bucket_name = '205915135'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

Inverted Index

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:

# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import *
TUPLE_SIZE = 6
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w,file_name):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    locs = [(file_name + lo[0], lo[1]) for lo in locs]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
#   raise NotImplementedError()
  tf=Counter(tokens)
  return [(token, (id, tf[token])) for token in tf if token not in all_stopwords]

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  #raise NotImplementedError()
  return sorted(unsorted_pl, key=lambda x: x[0])
  
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  #raise NotImplementedError()
  return postings.map(lambda x: (x[0], len(x[1])))
  
def partition_postings_and_write(postings ):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  #raise NotImplementedError()
  # partition the posting lists into buckets
  postings = postings.map(lambda x: (token2bucket_id(x[0]), x))
  # write out all posting lists in a bucket to disk
  postings = postings.groupByKey()
  postings = postings.map(lambda x: InvertedIndex.write_a_posting_list(x,bucket_name))
  # return the posting locations for each bucket
  return postings

In [ ]:
def calculate_term_total(postings):
    ''' Takes a posting list RDD and calculate the term total for each token.
    Parameters:
    -----------
    postings: RDD
        An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
    RDD
        An RDD where each element is a (token, term_total) pair.
    '''
    return postings.flatMapValues(lambda posting: [(doc_id, tf) for doc_id, tf in posting]).reduceByKey(lambda a,b: a+b)


In [ ]:
def DL(text, id):
    doc_length = len(list(filter(lambda x: x not in all_stopwords, [token.group() for token in RE_WORD.finditer(text.lower())])))
    return (id, doc_length)


In [ ]:
from pyspark.shell import spark

parquetFile = spark.read.parquet(*paths)

In [ ]:
parquetFile.count()


6348910

## Title


In [ ]:
doc_text_pairs_title = parquetFile.select("title", "id").rdd

In [ ]:
#title
word_counts_title = doc_text_pairs_title.flatMap(lambda x: word_count(x[0], x[1]))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)

In [ ]:
w2df_title = calculate_df(postings_title)
w2df_dict = w2df_title.collectAsMap()

w2terms = calculate_term_total(postings_title)
dict_term_total = w2terms.collectAsMap()


In [ ]:
# partition posting lists and write out
_ = partition_postings_and_write(postings_title).collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs_title = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_title[k].extend(v)

In [ ]:
# Create an instance of the InvertedIndex class
inverted_title = InvertedIndex()

# Add the posting locations for each token to the inverted index
inverted_title.posting_locs = super_posting_locs_title

# Add the term frequency (df) for each token to the inverted index
inverted_title.df = w2df_dict

# Add the total number of occurrences (term_total) for each token to the inverted index
inverted_title.term_total=dict_term_total

# Write the inverted index to a file in the specified directory with the given file name
inverted_title.write_index('.', 'title_index')


In [ ]:
index_title=pd.read_pickle('/home/dataproc/title_index.pkl')

In [ ]:
len(index_title.df)

1774265

## creat dict - id - title


In [ ]:
#creat dict - id - title
def create_title_dict(rdd):
    title_dict_rdd = rdd.map(lambda x: (x['id'], x['title']))
    return title_dict_rdd.collectAsMap()
# save dictionary to pickle file
title_dict = create_title_dict(doc_title_pairs)

with open("title_id.pkl", "wb") as file:
    pickle.dump(title_id, file, pickle.HIGHEST_PROTOCOL)

## BODY

In [ ]:
doc_text_pairs_body = parquetFile.select("text", "id").rdd

In [ ]:
word_counts_of_body = doc_text_pairs_body.flatMap(lambda x: word_count(x[0], x[1]))
postings_body = word_counts_of_body.groupByKey().mapValues(reduce_word_counts)
postings_filtered_body = postings_body.filter(lambda x: len(x[1])>50)

In [ ]:
w2df_body = calculate_df(postings_filtered_body)
w2df_body = w2df_body.collectAsMap()

w2termsbody = calculate_term_total(postings_filtered_body)
dict_term_total_body = w2termsbody.collectAsMap()


_ = partition_postings_and_write(postings_filtered_body).collect()
# collect all posting lists locations into one super-set
super_posting_locs_body = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_body[k].extend(v)

In [ ]:
# Create an instance of the InvertedIndex class
inverted_body = InvertedIndex()

# Add the posting locations for each token to the inverted index
inverted_body.posting_locs = super_posting_locs_body

# Add the term frequency (df) for each token to the inverted index
inverted_body.df = w2df_body

# Add the total number of occurrences (term_total) for each token to the inverted index
inverted_body.term_total = dict_term_total_body

# Add the document length for each document to the inverted index
inverted_body.DL=dl.collectAsMap()

# Write the inverted index to a file in the current directory with the given file name
inverted_body.write_index('.','body_index')


## DL OF BODY

In [ ]:
doc_text_DL = parquetFile.select("text", "id").rdd

In [ ]:
doc2len = doc_text_DL.map(lambda x:DL(x[0],x[1]))
dict_DL = doc2len.collectAsMap()

In [ ]:
import pickle
# Open a file to write the pickle to
with open("dict_DL.pkl", "wb") as pickle_file:
    # Write the dictionary to the file
    pickle.dump(dict_DL, pickle_file)

## Anchor Text

In [ ]:
doc_text_pairs_anchor_text=parquetFile.select("anchor_text", "id").rdd

In [ ]:
id2anchor=doc_text_pairs_anchor_text.flatMap(lambda x: x[0])
id2text=id2anchor.distinct().groupByKey().mapValues(lambda x: " ".join(x))

In [ ]:
word_counts = id2text.flatMap(lambda x: word_count(x[1], x[0]))

In [ ]:
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

In [ ]:
w2df_d = calculate_df(postings)
w2df_anchor = w2df_d.collectAsMap()

In [ ]:
w2termanchor = calculate_term_total(postings)
dict_term_total_anchor = w2termanchor.collectAsMap()

In [ ]:
_ = partition_postings_and_write(postings).collect()

In [ ]:
super_posting_locs_ANCOR = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_ANCOR[k].extend(v)

In [ ]:
# Create an instance of the InvertedIndex class
inverted_index_anchor = InvertedIndex()

# Add the posting locations for each token to the inverted index
inverted_index_anchor.posting_locs = super_posting_locs_ANCOR

# Add the term frequency (df) for each token to the inverted index
inverted_index_anchor.df = w2df_anchor

# Add the total number of occurrences (term_total) for each token to the inverted index
inverted_index_anchor.term_total = dict_term_total_anchor

# Write the inverted index to a file in the current directory with the given file name
inverted_index_anchor.write_index('.','anchor_index')

print('FINISH')

FINISH


## page rank

In [ ]:
# Put your `generate_graph` function here
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  # YOUR CODE HERE
  #raise NotImplementedError()
  # compute the directed graph generated by wiki links
  id_anchor = pages_links.mapValues(lambda item: [id for id, anchor_text in item])
  # get the edges
  edges = id_anchor.flatMap(lambda item: [(item[0], anchor_id) for anchor_id in item[1]])
  #distinct edges
  edges = edges.distinct()

  vertices = edges.flatMap(lambda item: [i for i in item]).map(lambda item: (item,))
  #distinct vertices
  vertices = vertices.distinct()
  return edges, vertices

In [ ]:
pages_links =  parquetFile.select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")

In [ ]:
import gzip
import pandas as pd

# Open the gzipped file and read the contents
with gzip.open('pagerank.csv.gz', 'rt') as f:
    file_content = f.read()

# Read the file content as a DataFrame
df = pd.read_csv(file_content)

# Create a dictionary using the values from column 0 as the keys and column 1 as the values
df_dict = df.set_index(0).to_dict()[1]

print(df_dict)

## page views

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)
# read in the counter
# with open(pv_clean, 'rb') as f:
#   wid2pv = pickle.loads(f.read())